![](https://fairmaster-konzept.com/wp-content/uploads/2018/03/bank-845x321.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Libraries

In [ ]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
import matplotlib.pyplot as plt


import data

In [ ]:
df=pd.read_csv("../input/churn-for-bank-customers/churn.csv")

*data review*

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

no missing values

In [ ]:
df['Exited'].plot.hist( rwidth=0.9,color='#FDCDA4')

**The sample is not balanced and this will affect the final result for sure

In [ ]:
df['Age'].plot.hist(grid=False, bins=100, rwidth=0.9,color='#FDCDA4')


In [ ]:
sns.countplot(x="Geography",hue="Exited",data=df)

In [ ]:
countplt, ax = plt.subplots(figsize = (16,8))
ax=sns.countplot(x="Age",hue="Exited",data=df)

# We note that the younger ages, most of them have left the bank, but this indicator will not be decisive because the sample is unbalanced

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['Geography'] = LE.fit_transform(df['Geography'])
df['Gender'] = LE.fit_transform(df['Gender'])


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df['Balance'] = scaler.fit_transform(df[['Balance']])
df['EstimatedSalary'] = scaler.fit_transform(df[['EstimatedSalary']])
df['Age'] = scaler.fit_transform(df[['Age']])

In [ ]:
df.corr()

In [ ]:
data=df.drop(["RowNumber","CustomerId","Surname"], axis=1)
data.head()

In [ ]:
features=data.drop('Exited',axis=1)
labele=data["Exited"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labele, test_size=0.33,random_state=42 )

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf =DecisionTreeClassifier(max_depth=5)

In [ ]:
from sklearn.ensemble import  VotingClassifier 

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
pred=clf.predict(X_test)

s=confusion_matrix(y_test, pred)
import seaborn as sn
import matplotlib.pyplot as plt
plt.figure(figsize = (16,9))
sn.heatmap(s, annot=True, fmt="d")

In [ ]:
from catboost import CatBoostClassifier

clf1 = CatBoostClassifier(
    iterations=2000, 
    learning_rate=0.01, 
    loss_function='CrossEntropy'
)


clf1.fit(X_train, y_train, 
        eval_set=(X_test, y_test), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf1.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())
clf1.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
pred=clf1.predict(X_test)

s=confusion_matrix(y_test, pred)
import seaborn as sn
import matplotlib.pyplot as plt
plt.figure(figsize = (16,9))
sn.heatmap(s, annot=True, fmt="d")

In [ ]:
eclf1 = VotingClassifier(estimators=[
('lr', clf), ('rf', clf1)],voting='hard',weights=[2,3],flatten_transform=True) 
eclf1 = eclf1.fit(X_train , y_train )
print(eclf1.score(X_test,y_test) )